In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

In [2]:
# Iniciando a sessão
driver_path = 'C:/Users/lucas/chromedriver.exe'
service = Service(executable_path=driver_path)

options = Options()
options.page_load_strategy = 'normal'
options.add_argument("--incognito")

driver = webdriver.Chrome(service=service, options=options)

In [3]:
# Navegando até a página
url = 'https://loterias.caixa.gov.br/Paginas/Mega-Sena.aspx'
driver.get(url)

In [4]:
# GOTO
search = driver.find_element(By.XPATH, '//*[@id="buscaConcurso"]')
start = 1500
search.send_keys(start)
search.send_keys(Keys.ENTER)

In [5]:
resultados = []

finish = 1
num_conc = start
count = 0
max_count = 2*(finish-start)
while num_conc > finish:
    count += 1
    if count == max_count:
        break
    time.sleep(1)

    try:
        # Números sorteados
        lista_dezenas = WebDriverWait(driver, timeout=10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="ulDezenas"]'))
        )
        dezenas = lista_dezenas.find_elements(By.TAG_NAME, 'li')
        dezenas = [ int(d.text) for d in dezenas ]

        # Número do concurso e data
        concurso = driver.find_element(By.XPATH, '//*[@id="wp_resultados"]/div[1]/div/h2/span')
        num_conc = concurso.text.split(' ')[1]
        num_conc = int(num_conc)

        # Guardar resultados
        dados = { f'n{i+1}': dezenas[i] for i in range(len(dezenas)) }
        dados['info_concurso'] = concurso.text
        resultados.append(pd.DataFrame(dados, index=[0]))

        # Ir para sorteio anterior
        sorteio_anterior = WebDriverWait(driver, timeout=10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="wp_resultados"]/div[1]/div/div[2]/ul/li[2]/a'))
        )
        sorteio_anterior.click()
    except:
        continue

df = pd.concat(resultados)

In [9]:
driver.quit()

In [8]:
# df.to_csv('data.csv', index=False)
df.to_csv('data.csv', index=False, mode='a', header=False)